In [ ]:
from const import *
import time
from web3 import Web3
from os import path
import json

web3 = Web3(Web3.HTTPProvider(BSC))
contract = web3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)

all_users = {}
all_tickets = {}
all_winners = {}

if path.exists("../"+USER_FILE):
    f = open("../"+USER_FILE, "r")
    all_users = json.load(f)
    f.close()

if path.exists("../"+TICKET_FILE):
    f = open("../"+TICKET_FILE, "r")
    all_tickets = json.load(f)
    f.close()

if path.exists("../"+WINNERS_FILE):
    f = open("../"+WINNERS_FILE, "r")
    all_winners = json.load(f)
    f.close()

for k, money in all_winners:
    wallet = all_users[k][0]
    nonce = web3.eth.getTransactionCount(OWNER)
    tx = contract.functions.senUSDT(wallet, money).buildTransaction({
        'chainId': CHAIN_ID, 'nonce': nonce, 'gas': 500000, 'gasPrice': 10000000000,
    })
    sign_tx = web3.eth.account.signTransaction(tx, PRIV_KEY)
    try:
        web3.eth.sendRawTransaction(sign_tx.rawTransaction)
        print(f"SendUSDT successful for {wallet}, {money}")
        del all_winners[k]
    except:
        print(f"SendUSDT failed for {wallet}, {money}")
    time.sleep(20)